### Ingest drivers.json file

##### Step 1 - Read the JSON file using the spark dataframe reader API

In [1]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

In [2]:
name_schema = StructType(fields=[StructField("forename", StringType(), True),
                                 StructField("surname", StringType(), True)
  
])

In [3]:
drivers_schema = StructType(fields=[StructField("driverId", IntegerType(), False),
                                    StructField("driverRef", StringType(), True),
                                    StructField("number", IntegerType(), True),
                                    StructField("code", StringType(), True),
                                    StructField("name", name_schema),
                                    StructField("dob", DateType(), True),
                                    StructField("nationality", StringType(), True),
                                    StructField("url", StringType(), True)  
])

In [4]:
drivers_df = spark.read \
.schema(drivers_schema) \
.json("/mnt/gualterformula1dl/raw/drivers.json")

##### Step 2 - Rename columns and add new columns
1. driverId renamed to driver_id  
1. driverRef renamed to driver_ref  
1. ingestion date added
1. name added with concatenation of forename and surname

In [5]:
from pyspark.sql.functions import col, concat, current_timestamp, lit

In [6]:
drivers_with_columns_df = drivers_df.withColumnRenamed("driverId", "driver_id") \
                                    .withColumnRenamed("driverRef", "driver_ref") \
                                    .withColumn("ingestion_date", current_timestamp()) \
                                    .withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname")))

##### Step 3 - Drop the unwanted columns
1. name.forename
1. name.surname
1. url

In [7]:
drivers_final_df = drivers_with_columns_df.drop(col("url"))

##### Step 4 - Write to output to processed container in parquet format

In [8]:
drivers_final_df.write.mode("overwrite").parquet("/mnt/gualterformula1dl/processed/drivers")